In [161]:
import pandas as pd
import numpy as np

In [162]:
store_inventory = pd.read_csv('Supplemental Order Store Data.csv')
forecast = pd.read_csv('Supplemental Order Forecast Data.csv')
dc_inventory = pd.read_excel('DC On Hand.xlsx')

In [163]:
store_inventory['store_in_transit_quantity_this_year'] = pd.to_numeric(store_inventory['store_in_transit_quantity_this_year'], errors='coerce').fillna(0)
store_inventory['store_in_warehouse_quantity_this_year'] = pd.to_numeric(store_inventory['store_in_warehouse_quantity_this_year'], errors='coerce').fillna(0)
store_inventory['store_on_hand_quantity_this_year'] = pd.to_numeric(store_inventory['store_on_hand_quantity_this_year'], errors='coerce').fillna(0)
store_inventory['store_on_order_quantity_this_year'] = pd.to_numeric(store_inventory['store_on_order_quantity_this_year'], errors='coerce').fillna(0)

# Now create the 'total_pipeline' column
store_inventory['total_pipeline'] = (
    store_inventory['store_in_transit_quantity_this_year'] + 
    store_inventory['store_in_warehouse_quantity_this_year'] + 
    store_inventory['store_on_hand_quantity_this_year'] + 
    store_inventory['store_on_order_quantity_this_year']
)

In [164]:
# Initialize an empty list to store the data
data = []

# getting unique list of items
items = store_inventory['vendor_stock_id'].unique()

for item in items:
    all_links_item_number = store_inventory[store_inventory['vendor_stock_id'] == item]['all_links_item_number'].values[0]

    # filtering the list of stores once we select the item
    inventory_filtered_item = store_inventory[store_inventory['vendor_stock_id'] == item]

    # sorting the stores based on the total pipeline ascending
    inventory_filtered_item = inventory_filtered_item.sort_values(by='total_pipeline', ascending=True)

    # getting the unique list of stores
    stores = inventory_filtered_item['store_number'].unique()

    total_shipped = 0
    for store in stores:
        # getting the total pipeline and vendor pack quantity for the selected variables
        inventory_filtered = inventory_filtered_item[inventory_filtered_item['store_number'] == store]
        total_pipeline = inventory_filtered['total_pipeline'].values[0]
        vendor_pack_quantity = inventory_filtered['vendor_pack_quantity'].values[0]
        max_shelf_quantity = inventory_filtered['max_shelf_quantity_this_year'].values[0]
        store_on_hand_quantity = inventory_filtered['store_on_hand_quantity_this_year'].values[0]
        dc = inventory_filtered['distribution_center_number'].values[0]
        idc = inventory_filtered['storage_distribution_center_number'].values[0]

        # getting the forecasted quantity for the selected variables
        forecast_filtered = forecast[(forecast['vendor_stock_id'] == item) & (forecast['store_nbr'] == store)]
        if not forecast_filtered.empty:
            forecasted_quantity = forecast_filtered['final_fcst_each_qty'].values[0]
        else:
            forecasted_quantity = 0  # Handle the case when no forecast is available

        # calculating the needed quantity for the item and store
        needed_quantity = total_pipeline - forecasted_quantity
        #max_shelf_minus_pipeline = max_shelf_quantity - total_pipeline
        if needed_quantity > 0:
            needed_quantity = 0
        else:
            needed_quantity = abs(needed_quantity)
            whse_packs_needed = np.ceil(needed_quantity / vendor_pack_quantity)
            needed_quantity = whse_packs_needed * vendor_pack_quantity
            #needed_quantity = needed_quantity - max_shelf_quantity
            if needed_quantity > max_shelf_quantity:
                needed_quantity = np.ceil(max_shelf_quantity / vendor_pack_quantity)
            else:
                #needed_quantity = needed_quantity
                needed_quantity = np.ceil(needed_quantity / vendor_pack_quantity)

        # Append the result to the list, including all_links_item_number
        data.append({
            'Item': item,
            'Store': store,
            'IDC': idc,
            'DC': dc,
            'Needed Quantity': needed_quantity,
            'All Links Item Number': all_links_item_number
        })
# Convert the list of dictionaries into a DataFrame
needed_quantity_df = pd.DataFrame(data)

# remove zeros
needed_quantity_df = needed_quantity_df[needed_quantity_df['Needed Quantity'] > 0]

In [165]:
# sort the dataframe based on item then needed quantity
needed_quantity_df = needed_quantity_df.sort_values(by=['All Links Item Number','IDC','Needed Quantity'], ascending=[True,False,False])

# SUM THE NEEDED QUANTITY FOR EACH ITEM
total_quantity_needed = needed_quantity_df.groupby(['All Links Item Number']).sum()
total_quantity_needed

,Item,Store,IDC,DC,Needed Quantity
All Links Item Number,,,,,
585320693,1785,4224,8980.0,6070.0,1.0
655181650,2932419,3219192,9247229.0,8321948.0,2103.0


In [166]:
needed_quantity_df

,Item,Store,IDC,DC,Needed Quantity,All Links Item Number
3942,1785,4224,8980.0,6070.0,1.0,585320693
886,2177,4255,8980.0,6024.0,3.0,655181650
1679,2177,638,8980.0,6070.0,3.0,655181650
1718,2177,877,8980.0,6070.0,3.0,655181650
2222,2177,2265,8980.0,6070.0,3.0,655181650
...,...,...,...,...,...,...
3167,2177,1588,6060.0,7033.0,1.0,655181650
3185,2177,2539,6060.0,6037.0,1.0,655181650
3216,2177,4438,6060.0,7026.0,1.0,655181650
3228,2177,2886,6060.0,7033.0,1.0,655181650


In [167]:
# Function to handle the row-by-row subtraction
def subtract_needed_quantity_by_item(needed_quantity_df, dc_inventory):
    # Loop through each unique item in the needed_quantity_df
    for item in needed_quantity_df['Item'].unique():

        # List of IDC numbers
        idc_numbers = [6060, 6061, 6088, 7042, 7067, 7078, 8980]

        # Dictionary comprehension to calculate the sum for each IDC
        idc_oh = {idc: dc_inventory.loc[
                    (dc_inventory['distribution_center_number'] == idc) & 
                    (dc_inventory['vendor_stock_id'] == str(item)),
                    'on_hand_warehouse_inventory_in_units_this_year'
                ].sum() 
                for idc in idc_numbers}

        # List of DC numbers
        dc_numbers = [6021, 6026, 6031, 6037, 7026, 7033, 6010, 6020, 6054, 7035, 7038, 6023, 6027, 6030, 6038, 
                    6080, 7034, 6012, 6016, 6019, 6035, 6036, 6068, 6094, 7036, 6006, 6011, 6017, 6018, 6048, 
                    6066, 6069, 6009, 6025, 6043, 6092, 7039, 6024, 6039, 6040, 6070, 7045]

        # Dictionary comprehension to calculate the sum for each DC
        dc_oh = {dc: dc_inventory.loc[
                    (dc_inventory['distribution_center_number'] == dc) & 
                    (dc_inventory['vendor_stock_id'] == str(item)),
                    'on_hand_warehouse_inventory_in_units_this_year'
                ].sum() 
                for dc in dc_numbers}  

        for idx, row in needed_quantity_df[needed_quantity_df['Item'] == item].iterrows():
            dc = row['DC']
            idc = row['IDC']
            needed_qty = row['Needed Quantity']

            # Check if we have enough in the total available quantity to subtract the needed quantity
            if dc_oh[dc] >= needed_qty:
                # If enough, subtract the needed quantity from available_quantity
                dc_oh[dc] -= needed_qty
                needed_quantity_df.at[idx, 'Fulfilled Quantity DC'] = needed_qty
                needed_quantity_df.at[idx, 'Fulfilled Quantity IDC'] = 0
            elif idc_oh[idc] >= needed_qty:
                # If enough, subtract the needed quantity from available_quantity
                idc_oh[idc] -= needed_qty
                needed_quantity_df.at[idx, 'Fulfilled Quantity IDC'] = needed_qty
                needed_quantity_df.at[idx, 'Fulfilled Quantity DC'] = 0
            else:
                # If not enough set fulfill to zero
                needed_quantity_df.at[idx, 'Fulfilled Quantity DC'] = 0
                needed_quantity_df.at[idx, 'Fulfilled Quantity IDC'] = 0

    return needed_quantity_df

# Apply the function to subtract from available inventory based on needed quantity
fulfilled_df = subtract_needed_quantity_by_item(needed_quantity_df, dc_inventory)

1785:6070.0:95.0
2177:6024.0:748.0
2177:6070.0:12.0
2177:6070.0:9.0
2177:6070.0:6.0
2177:6070.0:3.0
2177:6040.0:310.0
2177:6070.0:0.0
2177:6024.0:745.0
2177:6024.0:742.0
2177:6024.0:739.0
2177:6024.0:736.0
2177:6024.0:733.0
2177:7045.0:6112.0
2177:6024.0:731.0
2177:6039.0:260.0
2177:6039.0:258.0
2177:6039.0:256.0
2177:6024.0:729.0
2177:7045.0:6110.0
2177:6039.0:254.0
2177:6039.0:252.0
2177:6024.0:727.0
2177:6040.0:308.0
2177:6039.0:251.0
2177:7045.0:6109.0
2177:6039.0:250.0
2177:7045.0:6108.0
2177:6024.0:726.0
2177:6024.0:725.0
2177:6040.0:307.0
2177:6039.0:249.0
2177:6040.0:306.0
2177:6024.0:724.0
2177:6040.0:305.0
2177:6040.0:304.0
2177:6024.0:723.0
2177:6024.0:722.0
2177:6039.0:248.0
2177:6039.0:247.0
2177:6024.0:721.0
2177:6024.0:720.0
2177:6039.0:246.0
2177:6039.0:245.0
2177:6040.0:303.0
2177:6040.0:302.0
2177:7045.0:6107.0
2177:6039.0:244.0
2177:6039.0:243.0
2177:6040.0:301.0
2177:7045.0:6106.0
2177:7045.0:6105.0
2177:6040.0:300.0
2177:6040.0:299.0
2177:6039.0:242.0
2177:6040.0:2

In [168]:
fulfilled_df

,Item,Store,IDC,DC,Needed Quantity,All Links Item Number,Fulfilled Quantity DC,Fulfilled Quantity IDC
3942,1785,4224,8980.0,6070.0,1.0,585320693,1.0,0.0
886,2177,4255,8980.0,6024.0,3.0,655181650,3.0,0.0
1679,2177,638,8980.0,6070.0,3.0,655181650,3.0,0.0
1718,2177,877,8980.0,6070.0,3.0,655181650,3.0,0.0
2222,2177,2265,8980.0,6070.0,3.0,655181650,3.0,0.0
...,...,...,...,...,...,...,...,...
3167,2177,1588,6060.0,7033.0,1.0,655181650,1.0,0.0
3185,2177,2539,6060.0,6037.0,1.0,655181650,1.0,0.0
3216,2177,4438,6060.0,7026.0,1.0,655181650,1.0,0.0
3228,2177,2886,6060.0,7033.0,1.0,655181650,1.0,0.0


In [155]:
# drop the rows with nothing fulfilled
fulfilled_df = fulfilled_df[(fulfilled_df['Fulfilled Quantity DC'] > 0) | (fulfilled_df['Fulfilled Quantity IDC'] > 0)]
fulfilled_df

,Item,Store,IDC,DC,Needed Quantity,All Links Item Number,Fulfilled Quantity DC,Fulfilled Quantity IDC
3942,1785,4224,8980.0,6070.0,1.0,585320693,1.0,0.0
886,2177,4255,8980.0,6024.0,3.0,655181650,3.0,0.0
1679,2177,638,8980.0,6070.0,3.0,655181650,3.0,0.0
1718,2177,877,8980.0,6070.0,3.0,655181650,3.0,0.0
2222,2177,2265,8980.0,6070.0,3.0,655181650,3.0,0.0
...,...,...,...,...,...,...,...,...
3167,2177,1588,6060.0,7033.0,1.0,655181650,1.0,0.0
3185,2177,2539,6060.0,6037.0,1.0,655181650,1.0,0.0
3216,2177,4438,6060.0,7026.0,1.0,655181650,1.0,0.0
3228,2177,2886,6060.0,7033.0,1.0,655181650,1.0,0.0


In [156]:
# drop the columns that are not needed
fulfilled_df.drop(columns=['Needed Quantity'], inplace=True)
fulfilled_df

C:\Users\clayton\AppData\Local\Temp\ipykernel_13848\4209517079.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fulfilled_df.drop(columns=['Needed Quantity'], inplace=True)


,Item,Store,IDC,DC,All Links Item Number,Fulfilled Quantity DC,Fulfilled Quantity IDC
3942,1785,4224,8980.0,6070.0,585320693,1.0,0.0
886,2177,4255,8980.0,6024.0,655181650,3.0,0.0
1679,2177,638,8980.0,6070.0,655181650,3.0,0.0
1718,2177,877,8980.0,6070.0,655181650,3.0,0.0
2222,2177,2265,8980.0,6070.0,655181650,3.0,0.0
...,...,...,...,...,...,...,...
3167,2177,1588,6060.0,7033.0,655181650,1.0,0.0
3185,2177,2539,6060.0,6037.0,655181650,1.0,0.0
3216,2177,4438,6060.0,7026.0,655181650,1.0,0.0
3228,2177,2886,6060.0,7033.0,655181650,1.0,0.0


In [ ]:
fulfilled_df.to_csv('DC_push.csv', index=False)